### Document Loaders – Carregando dados com Langchain

#### Carregando PDFs

In [1]:
# importando a lib necessária
# para funcionar note também q é necessário ter a lib pypdf
from langchain_community.document_loaders import PyPDFLoader

In [2]:
# caminho para o arquivo
caminho = 'contrato_aluguel_jose_nilton.pdf'

In [3]:
# instanciando o carregador e fornecendo o caminho do arquivo pdf
loader = PyPDFLoader(caminho)
documentos = loader.load()

In [4]:
# veja que era um arquivo pdf pequeno
# o carregamento deu somente 5 páginas
len(documentos)

5

**Fazendo perguntas ao arquivo**

In [5]:
# importando a chain que permite fazermos as perguntas
from langchain.chains.question_answering import load_qa_chain

# importando o modelo
from langchain_openai.chat_models import ChatOpenAI

In [6]:
# instanciando o modelo para criar um chat
chat = ChatOpenAI()

In [7]:
# criando a chain
# o chain_type é um tipo de retrieval q será explicado mais adiante
chain = load_qa_chain(llm=chat, chain_type='stuff')

In [8]:
pergunta = "qual é o assunto do documento?"

In [9]:
# rodando a chain
r = chain.run(input_documents=documentos, question=pergunta)
print(r)

/home/kaio-peixoto/GitHub/asimov/aplicacoes-langchain/venv/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


O documento apresentado é um Contrato de Locação Residencial.


#### Carregando CSV

In [10]:
# o processo é bem parecido com o do pdf
# só temos q importar um loader diferente
from langchain_community.document_loaders.csv_loader import CSVLoader

In [11]:
# indicando o caminho pro arquivo
caminho_csv = 'Top 1000 IMDB movies.csv'

In [12]:
# carregando o csv
loader_csv = CSVLoader(caminho_csv)
docs_csv = loader_csv.load()

In [13]:
# vendo o conteudo pra certificar q trata-se do csv mesmo
docs_csv[0].page_content

': 0\nMovie Name: The Shawshank Redemption\nYear of Release: (1994)\nWatch Time: 142 min\nMovie Rating: 9.3\nMeatscore of movie: 81\nVotes: 34,709\nGross: $28.34M\nDescription: Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.'

In [14]:
# rodando a chain
pergunta = "tem aí o faturamento por filme? quem teve o menor?"
r = chain.run(input_documents=docs_csv[:100], question=pergunta)
print(r)

O faturamento dos filmes varia muito. O filme com o menor faturamento listado aqui é "A Great Dictator," de 1940, com um faturamento de $0.29M.


#### Carregando da Internet

##### YouTube

Para executar essa tarefa é necessário a existência de 2 arquivos ffmpeg.exe e ffprobe.exe. Aqui vamos só dar o exemplo em códigos, mas sem executar. Os arquivos ffmpeg estão disponíveis na pasta do curso da Asimov.

In [15]:
# importando as bibliotecas necessárias
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser

In [16]:
url = 'https://www.youtube.com/watch?v=pszz9Xk_T2c' # link para o video do youtube
save_dir = 'docs/youtube/' # diretorio onde o arquivo de audio transcrito ficará guardado
# construindo o loader
loader = GenericLoader(
    YoutubeAudioLoader([url], save_dir),
    OpenAIWhisperParser()
)

docs = loader.load()

ImportError: yt_dlp package not found, please install it with `pip install yt_dlp`

> Executando o codigo acima ele vai transcrever o audio e guardar a informação. Daí ela já estará disponivel para ser integrada ao modelo

##### URL

In [ ]:
from langchain_community.document_loaders.web_base import WebBaseLoader

In [ ]:
url = 'https://www.aihr.com/blog/decision-trees-hr-analytics/'
loader = WebBaseLoader(url)
docs_url = loader.load()

In [ ]:
# rodando a chain
pergunta = "qual o conteudo desse link q te passei?"
r = chain.run(input_documents=docs_url, question=pergunta)
print(r)

O conteúdo do link fornecido é um guia prático sobre como usar árvores de decisão em análises de RH. O texto explica o que é uma árvore de decisão, quando utilizá-la em análises de RH, terminologia associada, exemplos de casos de uso, e como construir uma árvore de decisão. Ele fornece detalhes sobre como as árvores de decisão podem ser uma ferramenta valiosa para analistas de RH, especialmente na descoberta de padrões complexos em dados de RH.


### Text Splitters

Aqui o objetivo é dividir o documento em trechos menores para fornecer ao LLM apenas pedaços relevantes da informação e assim ele processar. Isso garante mais agilidade e precisão no resultado final.

Mas como podemos manter a qualidade da informação se a dividimos em pedaços menores?

Vejamos esta frase de exemplo: "O novo carro da Fiat se chama Toro, tem 120 cavalos de potência e o preço sugerido é 135 mil reais."

Digamos q o split dessa frase ficaria da seguinte forma:

- "O novo carro da Fiat se"
- " chama Toro, tem 120 "
- "cavalos de potência e "
- "o preço sugerido é "
- "135 mil reais."

Aqui percebemos que os trechos separados individualmente não possuem valor. Sendo assim, seria de pouco valor para uma LLM, pois foi mal dividido.

Uma das técnicas para resolver isso é usar o parametro do overlapping. Esta técnica faz com que cada chunk tenha um pedaço do chunk anterior e do próximo. Dessa forma o LLM flui pela informação podendo fazer ligações.

Ficaria assim, por exemplo:

- "O novo carro da Fiat se"
- "da Fiat se chama Toro, tem 120 "
- "Toro, tem 120 cavalos de potência e "

In [17]:
# esse é o texto q vamos splittar
texto_grande = """
More and more organizations are turning to dashboards for monitoring performance and enabling data exploration. These user-friendly reporting tools offer a ton of advantages over older ways of doing things: they can dynamically update to display the latest information, link together multiple views of data, and often incorporate interactivity that lets users filter and zoom in on what they want to explore. 
"""

Existem formas diferentes de text splitter. Vamos explorar as mais utilizadas:

CharacterTextSplitter

In [18]:
from langchain_text_splitters import CharacterTextSplitter

In [19]:
# o tamanho do chunk (fatias) que vc deseja dividir
chunk_size = 50
# o tamanho da sobreposição. Geralmente um tamanho de 10 a 20% do chunk já funciona
chunk_overlap = 10

# criando o objeto q irá fazer o split
char_split = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator='' # o separator indica onde a divisão será feita
)

In [20]:
# vamos dar um exemplo de split com o abecedário
# o abecedario está disponivel importando a lib string
import string

In [21]:
# vamos criar nosso texto a ser dividido
# pegamos o abecedario e o replicamos 5 vezes
# veja q o tamanho do nosso texto ficou em 134 caracteres
texto = '-'.join(f'{string.ascii_lowercase}' for _ in range(5))
len(texto)

134

In [22]:
# agora fazendo o split do texto criado acima
# veja q o tamanho ficou em 4, sendo o chunksize de 50 
# e um total de 134 caracteres
splits = char_split.split_text(texto)
print(len(splits))
splits

4


['abcdefghijklmnopqrstuvwxyz-abcdefghijklmnopqrstuvw',
 'nopqrstuvwxyz-abcdefghijklmnopqrstuvwxyz-abcdefghi',
 '-abcdefghijklmnopqrstuvwxyz-abcdefghijklmnopqrstuv',
 'mnopqrstuvwxyz']

>> Veja acima como funciona o overlap. O segundo elemento da lista inicia com os 10 últimos caracteres da primeira lista, e assim sucessivamente. Ou seja, existe uma sobreposição entre o fim de um elemento da lista e inicio do próximo. A quantidade de caracteres que faz essa sobreposição é dada no chunk_overlap. E, claro, isso só não ocorre com o primeiro elemento, já q ele não tem antecessor.

**RecursiveCharacterTextSplitter**

Esse é o mais utilizado, ele permite o uso de vários separadores.

In [23]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [24]:
chunk_size = 50
chunk_overlap = 10

# criando o objeto q irá fazer o split
char_split = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)

In [25]:
# vamos aproveitar o mesmo exemplo usado anteriormente
texto = '-'.join(f'{string.ascii_lowercase}' for _ in range(5))
len(texto)

134

In [26]:
char_split.split_text(texto)

['abcdefghijklmnopqrstuvwxyz-abcdefghijklmnopqrstuvw',
 'nopqrstuvwxyz-abcdefghijklmnopqrstuvwxyz-abcdefghi',
 '-abcdefghijklmnopqrstuvwxyz-abcdefghijklmnopqrstuv',
 'mnopqrstuvwxyz']

Agora utilizando o argumento separators

Com este argumento o splitter cria uma ordem de prioridade para quebrar os chunks. No exemplo abaixo, primeiro ele quebra por ponto, depois por espaço vazio e depois por qualquer lugar do texto.

In [27]:
# criando o objeto q irá fazer o split
char_split = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=['.', ' ', '']
)

# geralmente apenas uns poucos marcadores de split
# são necessários. São eles: ['\n\n', '\n', ' ', '']

Vamos ver um exemplo com um texto grande, com mais caracteres do que o abecedário.

In [28]:
char_split.split_text(texto_grande)

['More and more organizations are turning to',
 'to dashboards for monitoring performance and',
 'and enabling data exploration',
 '. These user-friendly reporting tools offer a ton',
 'a ton of advantages over older ways of doing',
 'of doing things: they can dynamically update to',
 'update to display the latest information, link',
 'link together multiple views of data, and often',
 'and often incorporate interactivity that lets',
 'that lets users filter and zoom in on what they',
 'what they want to explore',
 '.']

**TokenTextSplitter**

Permite o fatiamento em tokens em vez de quantidade de caracteres.

In [29]:
from langchain_text_splitters import TokenTextSplitter

In [30]:
chunk_size = 50
chunk_overlap = 10

token_split = TokenTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

In [31]:
token_split.split_text(texto)

['abcdefghijklmnopqrstuvwxyz-abcdefghijklmnopqrstuvwxyz-abcdefghijklmnopqrstuvwxyz-abcdefghijkl',
 'uvwxyz-abcdefghijklmnopqrstuvwxyz-abcdefghijklmnopqrstuvwxyz']

>Agora veja como fica a divisão por token. O abecedário agora foi dividido em apenas duas partes. Isso porque aquela primeira parte toda já é equivalente a 50 tokens. Essa forma de split facilita nosso gerenciamento das APIs da OpenAI e outras, que contabilizam o fluxo de dados em tokens.

**MarkdownHeaderTextSplitter**

Podemos fazer o split de textos em Markdown. Existem vários marcadores nesse tipo de texto que podemos indicar para o split. Dessa forma fica mais fácil identificarmos quando trata-se de um texto principal (#), subtitulo (##), e etc.

In [32]:
markdown_text = """# Exemplo de titulo em markdown
## Aqui é um subtitulo
texto q fica dentro desse subtitulo
### Aqui outra hierarquia de header
Mais texto livre q fica dentro desta hierarquia
**e vamos de mais exemplos**
"""

In [33]:
# importando a biblioteca necessária
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [34]:
# primeiro criamos uma lista de tuplas
# onde terão os marcadores que farão o split

header_to_split_on = [
    ('#', 'Header 1'),
    ('##', 'Header 2'),
    ('###', 'Header 3'),
]

# criando o splitter
md_split = MarkdownHeaderTextSplitter(
    headers_to_split_on=header_to_split_on
)

In [35]:
# fazendo o split
splits = md_split.split_text(markdown_text)

In [36]:
splits

[Document(page_content='texto q fica dentro desse subtitulo', metadata={'Header 1': 'Exemplo de titulo em markdown', 'Header 2': 'Aqui é um subtitulo'}),
 Document(page_content='Mais texto livre q fica dentro desta hierarquia\n**e vamos de mais exemplos**', metadata={'Header 1': 'Exemplo de titulo em markdown', 'Header 2': 'Aqui é um subtitulo', 'Header 3': 'Aqui outra hierarquia de header'})]

>Veja como fica o output. Ele traz as hierarquias de título\subtitulo em metadata e traz seu conteúdo na variável page_content.

In [37]:
# vamos deixar esse output mais claro
# e vamos printar cada parte
for doc in splits:
    print(doc.page_content)
    print(doc.metadata)
    print("===========")

texto q fica dentro desse subtitulo
{'Header 1': 'Exemplo de titulo em markdown', 'Header 2': 'Aqui é um subtitulo'}
Mais texto livre q fica dentro desta hierarquia
**e vamos de mais exemplos**
{'Header 1': 'Exemplo de titulo em markdown', 'Header 2': 'Aqui é um subtitulo', 'Header 3': 'Aqui outra hierarquia de header'}


**Split de documentos**

Vamos fazer o split de um documento, em vez de textos simples como fizemos anteriormente.

In [38]:
# importando a biblioteca q carregará e nos ajudará a trabalhar com doc
from langchain_community.document_loaders import PyPDFLoader
# importando um splitter recursivo
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [39]:
chunk_size = 50
chunk_overlap = 10

# criando o objeto q irá fazer o split
# até aqui é identico ao q já fizemos anteriormente
char_split = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)

In [40]:
# caminho para o arquivo
caminho = 'contrato_aluguel_jose_nilton.pdf'

In [41]:
# carregando o arquivo
loader = PyPDFLoader(caminho)
docs = loader.load()

# veja que o tamanho é apenas 5, q é a quantidade de páginas
len(docs)

5

In [42]:
# agora vejamos depois de fazer o split
# fique atento que aqui o método é diferente, sendo split_documents
# em vez de split_text
doc_split = char_split.split_documents(docs)
len(doc_split)

227

### Embeddings - Transformando texto em vetores

Embeddings criam uma representação vetorial de um pedaço de texto. Ele transforma letras em números. Isso é útil, pois é assim que o modelo vai "pensar" de forma mais eficiente. Dessa forma ele pode buscar proximidades semanticas, etc. Os cálculos vetoriais é quem dão a "consciencia" e "discernimento" para o modelo.

O Langchain fornece uma classe de embedding que interage com os modelos de embedding fornecidos no mercado, ou seja, existem ebeddings da OpenAI, HuggingFace, Claude etc. O Langchain fornece uma interface padrão para todas elas.

In [43]:
from langchain_openai import OpenAIEmbeddings

In [44]:
# o modelo default para embedding é esse q está no argumento da chamada
# existem outros. Visite o site da OpenAI para detalhes
# vamos instanciar esse embedding da OpenAI
embedding_model = OpenAIEmbeddings(model='text-embedding-ada-002')

**Embedding de Documentos**

In [45]:
# vamos criar nosso modelo 
# veja que ele possui uma lista, onde ficarão os documentos
# inserimos 3 documentos, q no momento são apenas frases
embeddings = embedding_model.embed_documents(
    [
        'Eu amo frutas!',
        'Adoro comer maçã e morando no café da manhã',
        'Eu não gostei dessa piada.'
    ]
)

In [46]:
# temos 3 documentos
len(embeddings)

3

In [47]:
# vamos como ficou vetorizada a primeira frase
# aqui vemos os 10 primeiros vetores
embeddings[0][:10]

[-0.00657477715414702,
 -0.01645846854163768,
 0.00858595778552882,
 -0.018020670716986154,
 0.0003834389654492531,
 0.012147038753740848,
 0.0042468509564415,
 -0.016815192987670133,
 0.020923658738159195,
 -0.018254386091060882]

In [48]:
# como ficou a segunda...
embeddings[1][:10]

[0.0004073202309066271,
 0.0022691385116755613,
 0.010371467708254777,
 -0.010110052920248348,
 0.0008906330637083648,
 0.01609826783082426,
 -0.010243800151040828,
 -0.009763527358755165,
 0.0037874703983525005,
 -0.0054045923240678955]

In [49]:
# e a terceira...
embeddings[2][:10]

[-0.007192779163206384,
 0.010501820347042052,
 0.02364119459212685,
 0.005362824876373604,
 -0.01574063077795829,
 -0.00022685383203933346,
 0.0016772059794873417,
 -0.016877924626423603,
 0.01875325061971795,
 -0.015063094552185529]

In [50]:
# vamos fazer um for e analisar cada vetor
# primeiro vemos q o tamanho dos vetores é o mesmo (1536)
# isso acontece por causa do modelo escolhido da OpenAI, na hora de instanciar
# depois pegamos o valor máximo e o mínimo de cada lista de vetores
for emb in embeddings:
    print(len(emb), max(emb), min(emb))

1536 0.2231856932070314 -0.6502694359520379
1536 0.2404526844552327 -0.6606612596653819
1536 0.22455505498532602 -0.6345615833743852


In [51]:
# agora vamos fazer uma multiplicação vetorial utilizando o numpy
# para identificarmos as semelhanças semanticas entre os vetores
# é através desses cálculo que o modelo 'pensa' e sabe das semelhanças
# entre as frases

import numpy as np

In [52]:
# multiplicando a primeira e a segunda frase
# veja o número que obtemos
# quanto mais alto significa que as frases tem semânticas próximas
np.dot(embeddings[0], embeddings[1])

0.8840208063954538

In [53]:
# agora veja a comparação da frase 1 com a frase 3, o número é menor
np.dot(embeddings[0], embeddings[2])

0.7906236951124648

In [54]:
# mesma coisa entre as frases 2 e 3
np.dot(embeddings[1], embeddings[2])

0.7688656058169998

In [55]:
# vamos ver o conjunto todo, multiplicando todos vetores entre eles
for i in range(len(embeddings)):
    for j in range(len(embeddings)):
        print(round(np.dot(embeddings[i], embeddings[j]), 3), end=' | ')
    print()

1.0 | 0.884 | 0.791 | 
0.884 | 1.0 | 0.769 | 
0.791 | 0.769 | 1.0 | 


**Embedding Query**

Precisamos tbm fazer o embedding da pergunta do usuário ao modelo.

In [56]:
pergunta = "quais os alimentos mais saudáveis"
emb_query = embedding_model.embed_query(pergunta)
emb_query[:10]

[0.011808570584532625,
 -0.001554366564906917,
 0.003285103508179982,
 -0.0045724423097403,
 0.003168072813870247,
 0.0044142031522565305,
 -0.008650387718005114,
 -0.0342059548409562,
 -0.01714253627852646,
 0.003685645560653512]

In [57]:
# agora comparando com as frases criadas antes
# veja como a pergunta é próxima das frase 1 por exemplo
np.dot(emb_query, embeddings[1])

0.8124131758838706

In [58]:
# e veja com ela é mais distante da frase 3
np.dot(emb_query, embeddings[2])

0.7236383196137155

**Embedding com Huggingface**

In [59]:
from langchain_community.embeddings.huggingface import HuggingFaceBgeEmbeddings

In [60]:
model = 'all-MiniLM-L6-v2'

In [61]:
emb_model = HuggingFaceBgeEmbeddings(model_name=model)

/home/kaio-peixoto/GitHub/asimov/aplicacoes-langchain/venv/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/kaio-peixoto/GitHub/asimov/aplicacoes-langchain/venv/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [62]:
embeddings = emb_model.embed_documents(
    [
    'Eu amo frutas!',
    'Adoro comer maçã e morango no café da manhã',
    'A crise financeira de 2008 abalou o mundo.']
)

In [63]:
# vamos ver o conjunto todo, multiplicando todos vetores entre eles
for i in range(len(embeddings)):
    for j in range(len(embeddings)):
        print(round(np.dot(embeddings[i], embeddings[j]), 3), end=' | ')
    print()

1.0 | 0.348 | 0.413 | 
0.348 | 1.0 | 0.315 | 
0.413 | 0.315 | 1.0 | 


### VectorStores

Uma VectorStore faz o armazenamento de vetores e realiza a busca desses vetores. Depois de fazermos o embedding, como explicado na aula anterior, temos que guardá-los num lugar otimizado para trabalharmos com os vetores.

Temos várias soluções d VectorStores e aqui vamos estudar duas das mais utilizadas: Chroma e FAISS

**Chroma VectorStore**

In [64]:
# vamos fazer o processo desde o início
# ou seja, vamos carregar os documentos, splittar, fazer o embedding
# e guardar na VectorStore para retrieval

# importando as libs para carregar os docs
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [65]:
# carregando os documentos
caminho = 'contrato_aluguel_jose_nilton.pdf'
loader = PyPDFLoader(caminho)
docs = loader.load()

In [66]:
# agora vamos para o text splitting
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=30,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(docs)

**Criando a VectorStore**

In [67]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [68]:
embeddings_model = OpenAIEmbeddings()

In [69]:
diretorio = "files/chroma_vectorstore"

In [70]:
vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=diretorio
)